In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# /kaggle/input/google-smartphone-decimeter-challenge/train/2020-06-04-US-MTV-1/

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import plotly.express as px

## Training data Exploration

In [ ]:
train = pd.read_csv("/kaggle/input/google-smartphone-decimeter-challenge/baseline_locations_train.csv")
train.shape

In [ ]:
train.head()

In [ ]:
print(f"Unique number of collections : {train.collectionName.nunique()}")
coll_name = train.groupby("collectionName")['collectionName'].count().reset_index(name = 'counts')
fig = px.bar(coll_name, x='collectionName', y='counts', title = 'Data collection names')
fig.show()

In [ ]:
print(f"Unique number of Phone names : {train.phoneName.nunique()}")
phone_count = train.groupby('phoneName')['phoneName'].count().reset_index(name = 'counts')
fig = px.pie(phone_count, values='counts', names='phoneName', title='Phones used in data collection')
fig.show()

In [ ]:
print(f"Data collected using {train.phone.nunique()} phones")
ph_cnt = train.groupby('phone')['phone'].count().reset_index(name = 'counts')
ph_cnt

In [ ]:
latlon_trn = train[['latDeg','lngDeg']].round(3)
latlon_trn['counts'] = 1
latlon_trn = latlon_trn.groupby(['latDeg','lngDeg']).sum().reset_index()
latlon_trn

## Phone data Exploration

In [ ]:
for dirname, _, filenames in os.walk('../input/google-smartphone-decimeter-challenge/train/2020-06-04-US-MTV-1/Pixel4XL/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
def gnss_log_to_dataframes(path):
    print('Loading ' + path, flush=True)
    gnss_section_names = {'Raw','UncalAccel', 'UncalGyro', 'UncalMag', 'Fix', 'Status', 'OrientationDeg'}
    with open(path) as f_open:
        datalines = f_open.readlines()

    datas = {k: [] for k in gnss_section_names}
    gnss_map = {k: [] for k in gnss_section_names}
    for dataline in datalines:
        is_header = dataline.startswith('#')
        dataline = dataline.strip('#').strip().split(',')
        # skip over notes, version numbers, etc
        if is_header and dataline[0] in gnss_section_names:
            gnss_map[dataline[0]] = dataline[1:]
        elif not is_header:
            datas[dataline[0]].append(dataline[1:])

    results = dict()
    for k, v in datas.items():
        results[k] = pd.DataFrame(v, columns=gnss_map[k])
    # pandas doesn't properly infer types from these lists by default
    for k, df in results.items():
        for col in df.columns:
            if col == 'CodeType':
                continue
            results[k][col] = pd.to_numeric(results[k][col])

    return results


In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
dfs = gnss_log_to_dataframes('../input/google-smartphone-decimeter-challenge/train/2020-06-04-US-MTV-1/Pixel4XL/Pixel4XL_GnssLog.txt')
dfs['Raw'].head(5)

In [ ]:
dfs['UncalAccel'].head(5)

<font color = 'red'> <h1>Work in Progress </font>